# **NAIVE BAYES DOCUMENT CLASSIFIER**

### PPGIA / UFRPE

This code was developed as a task in the Text Mining class. 

### Filipe Gomes¹  
### Asaffe Matias²

**¹<filipe.avgomes@gmail.com>** <br>
**²<asaffe.m@gmail.com>**

## Library import:

In [1]:
import os

In [2]:
import nltk

In [3]:
import pandas as pd

In [4]:
from sklearn.metrics import confusion_matrix

In [5]:
from TextNormalizer import *

In [6]:
import math

In [7]:
import time

In [8]:
from sklearn.metrics import confusion_matrix

# **Main Code:**

## Labels:

### 1. class Model( )
Main class of this code. All the hard code is inside this class.

### 2. def __init___(self) 
Constructor.

### 3. def trainNBClassifier (self)
This method trains our classifier with the standard dataset. Here we extract parameters that our classifier will need to work on new documents.

### 4. def predict(self, file)
This method calculate the class probability in each document. It returns the maximum of the comparisson of classes probabilities inside a document.

### 5. def testNBClassifier(self)
After the classifier take the most likely class for a document, this method takes this information and put together inside the classification of the global dataset that is being classified. It returns all dataset already classified.

### 6. def getTrainingData(self):
This method gets the data that we will need to set parameters to our classifier.

### 7. def extractVocabulary(self):
Inside this method we extract the vocabulary that will be one of the parameters for the classifier training. There is some steps of data normalization on this method, as like punctuation and number removal, tokenization, among others.

### 8. def read_file(self,filename):
This method just read files from hard drive.

### 9. def save(self, model):  
Here we save our trained model inside a plain text file. This file will be saved on home folder with the name "NBClassifier".

### 10. def importModel(self, filename):
This method loads the trained model that was saved on file as NBClassifier.



In [65]:
class Model():
    def __init__(self):
        self.classes = {}
        self.logprob={} 
        self.vector_of_words=[]
        self.loglikelihood={}
        
            
    def trainNBClassifier(self):
         
        self.getTrainingData()    
        bigdoc={}
        
        for classe in self.classes:
            text=''
            
            number_of_documents = sum(map(lambda classes:int(classes),self.classes.values())) 
            #number_of_documents = self.classes[classe] + self.classes[classe]
            number_of_doc_in_class = self.classes[classe]
            
            
            self.logprob[classe] = math.log(number_of_doc_in_class/number_of_documents)
            #self.logprob[classe] = number_of_doc_in_class/number_of_documents

            self.vector_of_words = self.extractVocabulary()

            files=os.listdir('./dataset10/train/'+classe)
            for doc in files:
                text += self.read_file("./dataset10/train/"+classe+"/"+doc)
            
            bigdoc[classe]= text
            
            words_of_classe = nltk.tokenize.word_tokenize(bigdoc[classe])
            
            for word in self.vector_of_words:
                self.loglikelihood[word,classe] = math.log(bigdoc[classe].count(word) +1 / (len(words_of_classe)+len(self.vector_of_words)))
        print("NUMERO DOCS: ", number_of_documents)
        print("NUMBERO DOCS CLASSE: ", number_of_doc_in_class)
        print(self.classes)
        print(self.logprob)
        print(self.vector_of_words)
        print(self.loglikelihood)
                
        return (self.classes, self.logprob, self.vector_of_words, self.loglikelihood )
    
    def predict(self, file):
            
        words=[]
        probabilityPerClass={}
        doc= self.read_file(file)
        normalizer = TextNormalizer()
        text = normalizer.normalizing(doc)
        sentences = nltk.sent_tokenize(text)        
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)       
        
        for classe in self.classes:
            
            probabilityPerClass[classe] = self.logprob[classe]

            for word in words:
                if word in self.vector_of_words:
                    probabilityPerClass[classe]+= self.loglikelihood[word,classe]
            #print("CLASSE: ", probabilityPerClass)
            #print(len(self.vector_of_words))
        return max(probabilityPerClass, key=probabilityPerClass.get)
    
    def testNBClassifier(self):
        #path: caminho para o dataset test
        #buscar na pasta teste todos os arquivos e posteriormente chama predict()
        
        folders=os.listdir('./dataset10/test')
        y_test={}
        y_predict={}
        
        
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset10/test/'+classe))
            y_test[classe]=number_of_docs            
            y_predict[classe]=0
        
        
        t_inicial = time.time()  
        previsao = []
        real = []
        for classe in folders:  
            files=os.listdir('./dataset10/test/'+classe)

            
            for doc in files:
                if classe == 'pos':
                    real.append(1)
                else:
                    real.append(0)
                predict_class = self.predict('./dataset10/test/'+classe+'/'+doc)
                y_predict[predict_class]+=1
                #criando array para matriz de confusão
                if predict_class == 'pos':
                    previsao.append(1)
                else:
                    previsao.append(0)
        t_final = time.time()
        
        tempo_total = t_final - t_inicial
        print(y_test)
        print(y_predict)
        print("\n\033[1mELAPSED TIME:\033[0m " +str(tempo_total))
        
        return previsao, real
            
        
    def getTrainingData(self):
        folders=os.listdir('./dataset10/train')
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset10/train/'+classe))
            self.classes[classe]=number_of_docs        
          
    
    def extractVocabulary(self):
        bigtext=''
        folders=os.listdir('./dataset10/train')
        for classe in folders:            
            files=os.listdir('./dataset10/train/'+classe)
            for doc in files:
                bigtext += self.read_file("./dataset10/train/"+classe+"/"+doc)
        normalizer = TextNormalizer()
        bigtext = normalizer.normalizing(bigtext)
        sentences = nltk.sent_tokenize(bigtext)        
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            words= [w for w in words if w.isalpha()] # remove all tokens that are not alphabetic
        
        stopwords = open('stopwords.txt', 'r').read()
        word_list = [w for w in words if not w in stopwords]
           
        return set(word_list)
            
    
    def read_file(self,filename):
        with open (filename, 'r') as file:
            text = file.read()
        return text    
    
    def save(self, model):           
        f = open('NBclassifier','w')
        f.write(str(classifier))  
        f.close()
        
    def importModel(self, filename):
        model= self.read_file(filename)
        model= eval(model)
        self.classes = model[0]
        self.logprob = model[1]
        self.vector_of_words = model[2]
        self.loglikelihood = model[3]
        


## Training Call:

    The methods of training are called on this cell. Here the file with the trained classifier it will be saved on hard drive.

In [66]:
if __name__ == "__main__":
    model = Model()
    classifier = model.trainNBClassifier()
    model.save(classifier)   

NUMERO DOCS:  900
NUMBERO DOCS CLASSE:  450
{'neg': 450, 'pos': 450}
{'neg': -0.6931471805599453, 'pos': -0.6931471805599453}
{'martha', 'empath', 'hitchhiker', 'thoroughbreds', 'gillan', 'travesty', 'heavy', 'fonder', 'logjams', 'definitly', 'blankets', 'rightness', 'neglectful', 'climbs', 'innumerable', 'tunes', 'tapers', 'deservedly', 'ghost', 'sleezebag', 'prim', 'charter', 'movement', 'furnished', 'zooming', 'means', 'squandered', 'adults', 'ussr', 'wrecks', 'type', 'fireballs', 'superimposed', 'smartmouths', 'snuggling', 'gets', 'participants', 'exam', 'overwhelm', 'ugliness', 'mechanism', 'gyrating', 'seduce', 'ta', 'battlefield', 'perspectives', 'smoothes', 'moxxons', 'deceased', 'convention', 'marine', 'quips', 'lacey', 'craftsman', 'mermaid', 'angrier', 'medal', 'surroundings', 'delicious', 'intact', 'pitying', 'preparations', 'galas', 'schedules', 'underside', 'lay', 'morant', 'detract', 'archetypes', 'zwicks', 'alert', 'approved', 'schweig', 'fegan', 'witches', 'waldo', 'bu

# Output Session:
    In this session we have the classifier call, the output of the classifier, confusion matrix, and the some evaluation metrics.

## Classifier Call:

In [67]:

new_model = Model()

new_model.importModel('NBclassifier')

## Extracting the Confusion Matrix:

In [68]:
predict, real = new_model.testNBClassifier()

label = ["NEG", "POS"]
a = confusion_matrix(real, predict)
print("\n\033[1mMATRIZ DE CONFUSÃO:")
print("\t\t NEG POS")
for i in range(len(label)):
    print("\n\033[1m",label[i]," ->\t\033[0m", a[i])

{'neg': 50, 'pos': 50}
{'neg': 31, 'pos': 69}

ELAPSED TIME: 0.478405237197876

MATRIZ DE CONFUSÃO:
		 NEG POS

 NEG  ->	 [25 25]

 POS  ->	 [ 6 44]


## Calculating the Evaluation Metrics
In this session the following metrics were used:<br>
1. **Accuracy**
2. **Recall**
1. **Precision**
1. **F-Measure**

In [69]:
#calculando TP, TN, FP, FN:
correct = sum(1 for i in range(len(predict)) if predict[i] == real[i])
TP = sum(1 for i in range(len(predict)) if (predict[i] == real[i]) and predict[i] == 1)
TN = sum(1 for i in range(len(predict)) if (predict[i] == real[i]) and predict[i] == 0)
FP = sum(1 for i in range(len(predict)) if (predict[i] != real[i]) and predict[i] == 1)
FN = sum(1 for i in range(len(predict)) if (predict[i] != real[i]) and predict[i] == 0)

#ACCURACY
accuracy = (TP + TN) / len(predict)

#RECALL
recallTP = TP / (TP + FN)
recallTN = TN / (TN + FP)
mediarecall = (recallTP + recallTN) / 2

#PRECISION
precisionTP = TP / (TP + FP)
precisionTN = TN / (TN + FN)
precision = (precisionTN + precisionTP) / 2

#F-MEASURE
fscore = 2 * ((precision * mediarecall) / (precision + mediarecall))

print("\n\t \033[1m####### ACCURACY #######")
print("\n\033[1mACCURACY: \033[0m --> {0:.4f}".format(accuracy))

print("\n\t \033[1m ####### RECALL #######")

print("\n\033[1mRECALL POSITIVE: \033[0m --> {0:.4f}".format(recallTP))
print("\n\033[1mRECALL NEGATIVE: \033[0m --> {0:.4f}".format(recallTN))
print("\n\033[1mRECALL GLOBAL: \033[0m --> {0:.4f}".format(mediarecall))

print("\n\t \033[1m####### PRECISION #######")

print("\n\033[1mPRECISION POSITIVE: \033[0m --> {0:.4f}".format(precisionTP))
print("\n\033[1mPRECISION NEGATIVE: \033[0m --> {0:.4f}".format(precisionTN))
print("\n\033[1mPRECISION GLOBAL: \033[0m --> {0:.4f}".format(precision))

print("\n\t \033[1m ####### F-MEASURE #######")
print("\n\033[1mF-MEASURE: \033[0m --> {0:.4f}".format(fscore))


	 ####### ACCURACY #######

ACCURACY:  --> 0.6900

	  ####### RECALL #######

RECALL POSITIVE:  --> 0.8800

RECALL NEGATIVE:  --> 0.5000

RECALL GLOBAL:  --> 0.6900

	 ####### PRECISION #######

PRECISION POSITIVE:  --> 0.6377

PRECISION NEGATIVE:  --> 0.8065

PRECISION GLOBAL:  --> 0.7221

	  ####### F-MEASURE #######

F-MEASURE:  --> 0.7057
